In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
pd.set_option("display.max_columns", 500)

In [3]:
nppes_test = pd.read_csv('../data/npidata_pfile_20050523-20210207.csv', nrows=10000)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (17,34,76,80,84,87,89,90,91,93,94,95,97,98,99,101,102,103,105,106,171,175,178,179,181,182,183,185,186,189,190,191,193,194,197,198,201,205,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
nppes_test.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [233]:
# Examining Maeva's Code:
npidata_raw = pd.read_csv("../data/npidata_pfile_20050523-20210207.csv", nrows=1000)
npidata_raw.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [234]:
npidata = pd.concat([
    npidata_raw[['NPI']],
    # Entity Type Code: 1 = Provider (doctors, nurses, etc.) / 2 = Facility (Hospitals, Urgent Care, Doctors Offices) 
    npidata_raw[['Entity Type Code']],
    # Entity Name: Either First/Last or Organization or Other Organization Name contained in the following fields:
    npidata_raw.loc[:, 'Provider Organization Name (Legal Business Name)':'Provider Credential Text'],
    # Address: Business Practice Location (not mailing), contained in the following fields:
    npidata_raw.loc[:, 'Provider First Line Business Mailing Address':'Provider Business Mailing Address Postal Code'],
], axis=1)

npidata.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0


In [235]:
npi_taxonomy = pd.concat([
    npidata_raw[['NPI']],
    # The provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code*' columns
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Taxonomy Code_')]],
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Primary Taxonomy Switch_')]]
], axis=1)

npi_taxonomy.head()

,NPI,Healthcare Provider Taxonomy Code_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Primary Taxonomy Switch_15
0,1679576722,207X00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,207RC0000X,207RC0000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,251G00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,2085R0202X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,174400000X,207RH0003X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:
# Pivot from widet to long format
npi_taxonomy = pd.wide_to_long(
    npi_taxonomy,
    stubnames=['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch'],
    i=['NPI'], 
    j='primary_taxonomy_index',
    sep="_"
)

npi_taxonomy

,,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch
NPI,primary_taxonomy_index,,
1679576722,1,207X00000X,Y
1588667638,1,207RC0000X,N
1497758544,1,251G00000X,Y
1306849450,1,2085R0202X,Y
1215930367,1,174400000X,N
...,...,...,...
1417950635,15,NaN,NaN
1326041542,15,NaN,NaN
1235132457,15,NaN,NaN


In [237]:
# Only keep the primary taxonomy
npi_taxonomy = npi_taxonomy[npi_taxonomy['Healthcare Provider Primary Taxonomy Switch'] == 'Y']

npi_taxonomy

,,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch
NPI,primary_taxonomy_index,,
1679576722,1,207X00000X,Y
1497758544,1,251G00000X,Y
1306849450,1,2085R0202X,Y
1023011178,1,251G00000X,Y
1932102084,1,207RC0000X,Y
...,...,...,...
1699778829,7,207X00000X,Y
1811990047,8,333600000X,Y
1619970845,8,3336H0001X,Y


In [238]:
# Housekeeping
npi_taxonomy = npi_taxonomy.reset_index()\
    .drop(columns=['primary_taxonomy_index', 'Healthcare Provider Primary Taxonomy Switch'])\
    .rename({ 'Healthcare Provider Taxonomy Code': 'primary_taxonomy' }, axis=1)

npi_taxonomy

,NPI,primary_taxonomy
0,1679576722,207X00000X
1,1497758544,251G00000X
2,1306849450,2085R0202X
3,1023011178,251G00000X
4,1932102084,207RC0000X
...,...,...
932,1699778829,207X00000X
933,1811990047,333600000X
934,1619970845,3336H0001X
935,1164425336,3336H0001X


In [239]:
npidata = npidata.merge(
    npi_taxonomy,
    how='left',
    on='NPI'
)

npidata.head()


,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,primary_taxonomy
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0,207X00000X
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X


In [240]:
npidata = npidata.reset_index().rename({
    'NPI': 'npi',
    'Entity Type Code': 'entity_type_code',
    'Provider Organization Name (Legal Business Name)': 'provider_org_name',
    'Provider Last Name (Legal Name)': 'provider_last_name',
    'Provider First Name': 'provider_first_name',
    'Provider Middle Name': 'provider_middle_name',
    'Provider Name Prefix Text': 'provider_name_prefix',
    'Provider Name Suffix Text': 'provider_name_suffix',
    'Provider Credential Text': 'provider_credential',
    'Provider First Line Business Mailing Address': 'provider_business_address_1',
    'Provider Second Line Business Mailing Address': 'provider_business_address_2',
    'Provider Business Mailing Address City Name': 'provider_business_city',
    'Provider Business Mailing Address State Name': 'provider_business_state',
    'Provider Business Mailing Address Postal Code': 'provider_business_zip'
}, axis=1)

npidata.head()

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy
0,0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0,207X00000X
1,1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X


### Next, download the taxonomy code to classification crosswalk 
from the National Uniform Claim Committee (https://www.nucc.org/index.php/code-sets-mainmenu-41/provider-taxonomy-mainmenu-40/csv-mainmenu-57). Using the primary taxonomy code, match each provider to a classification (from the Classification column).

In [5]:
nucc = pd.read_csv("../data/nucc_taxonomy_210.csv")

In [6]:
nucc.shape

(865, 10)

In [7]:
nucc.head()

,Code,Grouping,Classification,Specialization,Definition,Effective Date,Deactivation Date,Last Modified Date,Notes,Display Name
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Multi-Specialty Group
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Single Specialty Group
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,4/1/2003,NaN,7/1/2007,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,4/1/2003,NaN,NaN,NaN,Allergy Physician
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,4/1/2003,NaN,NaN,NaN,Clinical & Laboratory Immunology (Allergy & Im...


In [8]:
# Keep only the columns we need.
nucc = nucc[['Code', 'Classification']]
nucc.head()

,Code,Classification
0,193200000X,Multi-Specialty
1,193400000X,Single Specialty
2,207K00000X,Allergy & Immunology
3,207KA0200X,Allergy & Immunology
4,207KI0005X,Allergy & Immunology


In [9]:
# Merge to npidata, matching on primary_taxonomy/Code
npidata = npidata.merge(nucc, how='left', left_on = 'primary_taxonomy', right_on = 'Code', copy=False)
npidata = npidata.drop('Code', axis=1)

NameError: name 'npidata' is not defined

In [246]:
npidata.head()

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy,Classification
0,0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,PO BOX 2168,NaN,KEARNEY,NE,688482168.0,207X00000X,Orthopaedic Surgery
1,1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X,Internal Medicine
2,2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X,"Hospice Care, Community Based"
3,3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X,Radiology
4,4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X,Internal Medicine


### Finally, you need the Zip code to CBSA crosswalk 
from here: https://www.huduser.gov/portal/datasets/usps_crosswalk.html. Match each provider to a CBSA using the Business Zip code. Note that a zipcode can be associated to multiple CBSAs, so match with the CBSA with the highest TOT_RATIO. Note that the zipcodes in the nppes dataset are either 5 or 9 digits, and be mindful that leading zeros might be dropped when reading the dataset into a dataframe.

In [247]:
zips = pd.read_excel("../data/ZIP_CBSA_122017.xlsx", converters={'zip': lambda x: str(x)})

In [248]:
zips.head()

,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00612,11640,0.996207,0.999588,0.995662,0.996515
1,00627,11640,1.000000,1.000000,1.000000,1.000000
2,00638,41980,1.000000,1.000000,1.000000,1.000000
3,00676,10380,1.000000,1.000000,1.000000,1.000000
4,00719,41980,1.000000,1.000000,1.000000,1.000000


In [249]:
# Filter to zips within the Nashville cbsa
zips = zips[zips['cbsa'] == 34980]

In [250]:
# Each zip code can be in multiple counties, though we only want to match zips to the county with the greatest representation.
# To do this, groupby county, and keep only the rows with the max tot_ratio.
zips = zips.groupby('zip')['tot_ratio'].max().reset_index()
zips

,zip,tot_ratio
0,37010,0.411630
1,37011,1.000000
2,37012,0.216487
3,37013,1.000000
4,37014,1.000000
...,...,...
158,38563,1.000000
159,38567,0.941315
160,38569,0.292111
161,38575,0.001708


In [251]:
# Create a list of counties we will use as a filter for the nationwide npidata
zips_list = zips['zip'].to_list()
zips_list

['37010',
 '37011',
 '37012',
 '37013',
 '37014',
 '37015',
 '37016',
 '37018',
 '37020',
 '37022',
 '37024',
 '37025',
 '37026',
 '37027',
 '37029',
 '37030',
 '37031',
 '37032',
 '37033',
 '37035',
 '37036',
 '37037',
 '37043',
 '37046',
 '37048',
 '37049',
 '37051',
 '37052',
 '37055',
 '37056',
 '37057',
 '37060',
 '37062',
 '37063',
 '37064',
 '37065',
 '37066',
 '37067',
 '37068',
 '37069',
 '37070',
 '37071',
 '37072',
 '37073',
 '37074',
 '37075',
 '37076',
 '37077',
 '37080',
 '37082',
 '37083',
 '37085',
 '37086',
 '37087',
 '37088',
 '37089',
 '37090',
 '37091',
 '37095',
 '37098',
 '37101',
 '37110',
 '37115',
 '37116',
 '37118',
 '37119',
 '37121',
 '37122',
 '37127',
 '37128',
 '37129',
 '37130',
 '37131',
 '37132',
 '37133',
 '37135',
 '37137',
 '37138',
 '37140',
 '37141',
 '37143',
 '37145',
 '37146',
 '37148',
 '37149',
 '37150',
 '37151',
 '37152',
 '37153',
 '37160',
 '37165',
 '37166',
 '37167',
 '37171',
 '37172',
 '37174',
 '37179',
 '37180',
 '37181',
 '37184',


### Filter npidata to only where zipcodes are within Nashville cbsa

In [255]:
# Start by converting provider_business_zip	to string and indexing to first 5 digits
npidata['provider_business_zip5'] = [str(i)[0:5] for i in npidata['provider_business_zip']]

In [257]:
# Filter rows to 'provider_business_zip5' being in zips_list
npidata_nashville = npidata[npidata['provider_business_zip5'].isin(zips_list)]

In [260]:
npidata_nashville.head()

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy,Classification,provider_business_zip5
67,67,1003819046,1.0,NaN,NYLANDER,BARBARA,H,NaN,NaN,M.D.,3024 BUSINESS PARK CIR,NaN,GOODLETTSVILLE,TN,370723132.0,207VG0400X,Obstetrics & Gynecology,37072
76,76,1750384780,1.0,NaN,PERRIGIN,JULIE,A,DR.,NaN,MD,PO BOX 778,NaN,DICKSON,TN,370560778.0,207Q00000X,Family Medicine,37056
105,105,1073516001,1.0,NaN,ROSS,DAVID,L,DR.,NaN,MD,127 CRESTVIEW PARK DR,NaN,DICKSON,TN,370552850.0,207R00000X,Internal Medicine,37055
117,117,1780687830,1.0,NaN,MANI,VENK,NaN,DR.,NaN,MD,127 CRESTVIEW PARK DR,NaN,DICKSON,TN,370552850.0,207ZC0500X,Pathology,37055
171,171,1437152485,1.0,NaN,MORGAN,LISA,BROOKS,NaN,NaN,MD,2201 MURPHY AVE STE 407,NaN,NASHVILLE,TN,372031864.0,207V00000X,Obstetrics & Gynecology,37203


In [258]:
npidata.shape

(1000, 18)

In [259]:
npidata_nashville.shape

(25, 18)

### CSVs for Matthew

In [12]:
taxonomy = pd.read_csv("../data/nucc_taxonomy_210.csv")
taxonomy = taxonomy[['Code', 'Grouping', 'Classification', 'Specialization']]
taxonomy.columns = ['taxonomy_code', 'grouping', 'classification', 'specialization']
taxonomy.to_csv('taxonomy.csv')

In [16]:
zips = pd.read_excel("../data/ZIP_CBSA_122017.xlsx", converters={'zip': lambda x: str(x)})
zips = zips[zips['cbsa'] == 34980]
zips.to_csv('zips.csv')

### Ignore the rest for now.

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [ ]:
db = sqlite3.connect('../data/DocGraph_Hop_Teaming_2017.sqlite')

Now, we can chunk through the data and for each row, add the rows to a table in our sqlite database. 
To keep track of how much progress has been made, we can use the `tqdm` library.

In [ ]:
for chunk in tqdm(pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('calls', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

To speed up queries which use a specific column, we can create an **index** on that column. This causes the database to store that column in a way that helps it to retrieve rows quicker.

In [ ]:
db.execute('CREATE INDEX tencode_description ON calls(tencode_description)')

Finally, we should close our database connection.

In [ ]:
db.close()

Now, let's see how long it takes to find all rows corresponding to 'SHOTS FIRED'.

In [ ]:
db = sqlite3.connect('data/police_calls.sqlite')

query = "SELECT * FROM calls WHERE tencode_description = 'SHOTS FIRED'"

shots_sqlite = pd.read_sql(query, db)

db.close()

In [ ]:
shots_sqlite.head()

Add a table named "incidents" to your police_calls.sqlite database and load in the Metro_Nashville_Police_Department_Incidents.csv file

In [ ]:
# Your code here
db = sqlite3.connect('data/police_calls.sqlite')

for chunk in tqdm(pd.read_csv('data/Metro_Nashville_Police_Department_Incidents.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('incidents', db, if_exists = 'append', index = False)            # Append the chunk to a calls table
    
db.close()

If we want to match calls to incidents, we can use the complaint_number column from the calls database and the incident_number column from the incidents database.

To speed up this process, we can created indexes on these two columns.

In [ ]:
db = sqlite3.connect('data/police_calls.sqlite')

db.execute('CREATE INDEX complaint_number ON calls(complaint_number)')
db.execute('CREATE INDEX incident_number ON incidents(incident_number)')

db.close()

Now, let's grab all SHOTS FIRED calls for which there is an associated incident and bring in the incident information.

In [ ]:
query = """
    SELECT * 
    FROM calls AS c 
    JOIN incidents AS i
    ON c.complaint_number = i.incident_number
    WHERE tencode_description = 'SHOTS FIRED'
"""

In [ ]:
with sqlite3.connect('data/police_calls.sqlite') as db:
    shots_sqlite = pd.read_sql(query, db)

In [ ]:
shots_sqlite

## Chunking

In [ ]:
metro_calls = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', 
                          nrows = 10000)

In [ ]:
metro_calls.head()

Let's say that our goal is to find all incidents where the Tencode Description was 'SHOTS FIRED'.

One thing we could try is to use the chunksize argument in our pd.read_csv call. What this does is to create an iterable which returns just the specified number of rows at a time.

Iterating through a file using chunks can look like this, but can also be structured differently (for example, using a list comprehension).

``` 
chunks = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)

for chunk in chunks:
    # Do something 
```

Here is what we need to do:

1. Create an iterable by using the chunksize argument.

2. For each chunk, filter to just the rows where the 'Tencode Description' column is 'SHOTS FIRED'. Store these rows.

3. Concatenate all the results together into a single dataframe.

In [ ]:
chunks = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)
chunky_df = pd.DataFrame([])

for chunk in chunks:
    chunk_bit = chunk[chunk['Tencode Description'] == 'SHOTS FIRED']
    chunky_df = chunky_df.append(chunk_bit)

In [ ]:
chunky_df.shape

In [ ]:
#list comprehension:
shots_fired = pd.concat(
    [chunk[chunk['Tencode Description'] == 'SHOTS FIRED'] for chunk in pd.read_csv(
        'data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)])

In [ ]:
shots_fired.shape